In [1]:
import darshan
import numpy
import os.path
import pandas

In [2]:
filename="ImageProcessing/D2024-04-18_15:54:02_R1_W8/agueroud_python3.10_id56357-56357_4-18-57271-14829777830687840415_1.darshan"
os.path.isfile(filename)

True

Exploratory tests 1

In [3]:
header="ImageProcessing/D2024-04-18_15:54:02_R1_W8/"
filename1="agueroud_python3.10_id56357-56357_4-18-57271-14829777830687840415_1.darshan" # lorge file
filename2="agueroud_python3.10_id11355-11355_4-18-57271-17993944196753836403_1.darshan" # lorge file
filename3="agueroud_mpiexec_id12762-12762_4-18-57269-13840249305489711412_1.darshan"
filename4="agueroud_python3.10_id11346-11346_4-18-57271-15569483023332165605_1.darshan"

In [4]:
with darshan.DarshanReport(header+filename1,read_all=False) as report:
    print("metadata: ", report.metadata)
    print(report.info())
    # NOTE: have to actually _read_ the records before you can access them.
    report.mod_read_all_records('POSIX', dtype='numpy', warnings=True)
    
    
    df = report.records['POSIX'].to_df()
    print(df['counters'])
    
    

metadata:  {'job': {'uid': 37993, 'start_time_sec': 1713455671, 'start_time_nsec': 934386245, 'end_time_sec': 1713455882, 'end_time_nsec': 893607402, 'nprocs': 1, 'jobid': 56357, 'run_time': 210.95922112464905, 'log_ver': '3.41', 'metadata': {'lib_ver': '3.4.4', 'h': 'romio_no_indep_rw=true;cb_nodes=4'}}, 'exe': '/home/agueroudji/spack/var/spack/environments/mofkadask/.spack-env/._view/ycbghmgvq7z34ridg4nntzus2jsgkcos/bin/python3.10 -c from multiprocessing.spawn import spawn_main; spawn_main(tracker_fd=11, pipe_handle=17) --multiprocessing-fork '}
Filename:       ImageProcessing/D2024-04-18_15:54:02_R1_W8/agueroud_python3.10_id56357-56357_4-18-57271-14829777830687840415_1.darshan
Executable:     /home/agueroudji/spack/var/spack/environments/mofkadask/.spack-env/._view/ycbghmgvq7z34ridg4nntzus2jsgkcos/bin/python3.10 -c from multiprocessing.spawn import spawn_main; spawn_main(tracker_fd=11, pipe_handle=17) --multiprocessing-fork 
Times:          2024-04-18 11:54:31 to 2024-04-18 11:58:02

In [2]:
# open a Darshan log file and read all data stored in it
with darshan.DarshanReport(filename, read_all=True) as report:

    # print the metadata dict for this log
    print("metadata: ", report.metadata)
    # print job runtime and nprocs
    print("run_time: ", report.metadata['job']['run_time'])
    print("nprocs: ", report.metadata['job']['nprocs'])

    # print modules contained in the report
    print("modules: ", list(report.modules.keys()))

    # export POSIX module records to DataFrame and print
    posix_df = report.records['POSIX'].to_df()
    print("POSIX df: ", posix_df)

: 

Exploratory tests 2 - Find differences between different types of darshan logs

In [ ]:
def read_file(filename, skip_dxt_posix=True) :
    output = {}
    # have to not read_all in case it crashes out in certain modules.
    with darshan.DarshanReport(filename, read_all=False) as report:
        output["metadata"] = report.metadata
        modules = list(report.modules.keys())

        output["modules"] = modules
        expected_modules = ["POSIX", "LUSTRE", "STDIO", "DXT_POSIX", "HEATMAP", "MPI-IO", "DXT_MPIIO"]

        if "POSIX" in modules :
            report.mod_read_all_records("POSIX",dtype="numpy")
            output["POSIX"] = report.records["POSIX"].to_df() 
        
        if "LUSTRE" in modules :
            report.mod_read_all_lustre_records(dtype="numpy")
            output["LUSTRE"] = report.records["LUSTRE"].to_df()

        if "STDIO" in modules :
            report.mod_read_all_records("STDIO", dtype="numpy")
            output["STDIO"] = report.records["STDIO"].to_df()

        # DXT_POSIX causes a kernel crash _only in amal's data_.
        if "DXT_POSIX" in modules and not skip_dxt_posix :
            report.mod_read_all_dxt_records("DXT_POSIX",dtype="numpy")
            output["DXT_POSIX"] = report.records["DXT_POSIX"].to_df()

        # Received message: Skipping. Currently unsupported: HEATMAP in mod_read_all_records().
        # if "HEATMAP" in modules :
        #     report.mod_read_all_records("HEATMAP", dtype="numpy")
        #     output["HEATMAP"] = report.records["HEATMAP"].to_df()

        if "MPI-IO" in modules :
            report.mod_read_all_records("MPI-IO", dtype="numpy")
            output["MPI-IO"] = report.records["MPI-IO"].to_df()

        if "DXT_MPIIO" in modules :
            report.mod_read_all_dxt_records("DXT_MPIIO", dtype="numpy")
            output["DXT_MPIIO"] = report.records["DXT_MPIIO"].to_df()

        for m in modules :
            if m not in expected_modules :
                print("unexpected module found: %s" % m)

    return output

: 

In [ ]:
header = "ImageProcessing/D2024-04-18_15:54:02_R1_W8"

# get all the darshan logs
files = [f for f in os.listdir(header) if os.path.isfile(os.path.join(header, f))]
files = [f for f in files if os.path.splitext(f)[1] == ".darshan" or os.path.splitext(f)[1] == ".darshan_partial"]

# get the unique executable prefixes
file_executables = list(set([f.split("_")[1] for f in files]))

# process each file and group by executable prefix
by_executable = {}
for fe in file_executables :
    by_executable[fe] = []

prefix_mod_counts = {}
for fe in file_executables :
    prefix_mod_counts[fe] = {"total":0}

for f in files :
    # darshan_partial throws an error when you try to load it at all.
    if os.path.splitext(f)[1] == ".darshan" :
        fe = f.split("_")[1]
        output = read_file(os.path.join(header,f), skip_dxt_posix=False)

        by_executable[fe].append(output)

        # get some statistics about executable prefix module representation
        prefix_mod_counts[fe]["total"] = prefix_mod_counts[fe]["total"] + 1
        for m in output['modules'] :
            if m not in prefix_mod_counts[fe].keys() :
                prefix_mod_counts[fe][m] = 1
            else :
                prefix_mod_counts[fe][m] = prefix_mod_counts[fe][m] + 1

print(prefix_mod_counts)

In [13]:
by_executable

{'uname': [{'metadata': {'job': {'uid': 37993,
     'start_time_sec': 1713455673,
     'start_time_nsec': 456646316,
     'end_time_sec': 1713455673,
     'end_time_nsec': 459439892,
     'nprocs': 1,
     'jobid': 11399,
     'run_time': 0.0027937889099121094,
     'log_ver': '3.41',
     'metadata': {'lib_ver': '3.4.4',
      'h': 'romio_no_indep_rw=true;cb_nodes=4'}},
    'exe': 'uname -p '},
   'modules': ['POSIX'],
   'POSIX': {'counters':    rank                   id  POSIX_OPENS  POSIX_FILENOS  POSIX_DUPS  \
    0     0  7238257241479193519            1              1           0   
    
       POSIX_READS  POSIX_WRITES  POSIX_SEEKS  POSIX_STATS  POSIX_MMAPS  ...  \
    0            0             0            0            0           -1  ...   
    
       POSIX_ACCESS3_ACCESS  POSIX_ACCESS4_ACCESS  POSIX_ACCESS1_COUNT  \
    0                     0                     0                    0   
    
       POSIX_ACCESS2_COUNT  POSIX_ACCESS3_COUNT  POSIX_ACCESS4_COUNT  \
    0   

# Exploratory Tests 3 - Compare to example darshan logs from the darshan-hpc repository.

Discoveries:
- If a record has a `DXT_POSIX`, it should be readable. Why does Amal's cause a kernel crash?
- Two new types of modules I haven't implemented yet:  `MPI-IO`, `DXT_MPIIO`

Need to check:
- Do these throw an error message for HEATMAP too?

In [24]:
pre_header = "ex_darshan_logs"

# header = "empty_log"
# header = "stdio_no_posix"
header = "partial_data_dxt"

header = os.path.join(pre_header, header)

In [44]:

# get all the darshan logs
files = [f for f in os.listdir(header) if os.path.isfile(os.path.join(header, f))]
files = [f for f in files if os.path.splitext(f)[1] == ".darshan" or os.path.splitext(f)[1] == ".darshan_partial"]

# get the unique executable prefixes
file_executables = list(set([f.split("_")[1] for f in files]))

# process each file and group by executable prefix
by_executable = {}
for fe in file_executables :
    by_executable[fe] = []

prefix_mod_counts = {}
for fe in file_executables :
    prefix_mod_counts[fe] = {"total":0}

for f in files :
    # darshan_partial throws an error when you try to load it at all.
    if os.path.splitext(f)[1] == ".darshan" :
        fe = f.split("_")[1]
        output = read_file(os.path.join(header,f), False)

        by_executable[fe].append(output)

        # get some statistics about executable prefix module representation
        prefix_mod_counts[fe]["total"] = prefix_mod_counts[fe]["total"] + 1
        for m in output['modules'] :
            if m not in prefix_mod_counts[fe].keys() :
                prefix_mod_counts[fe][m] = 1
            else :
                prefix_mod_counts[fe][m] = prefix_mod_counts[fe][m] + 1

print(prefix_mod_counts)
# print(by_executable['data'][0]['DXT_POSIX'])
# print(by_executable['data'][0]['POSIX'])

{'data': {'total': 1, 'POSIX': 1, 'MPI-IO': 1, 'STDIO': 1, 'DXT_POSIX': 1, 'DXT_MPIIO': 1}}


In [46]:
by_executable['data'][0]['DXT_MPIIO']

[{'id': 15574190512568163195,
  'rank': 0,
  'hostname': 'carns-x1-7g',
  'write_count': 0,
  'read_count': 65529,
  'write_segments': Empty DataFrame
  Columns: []
  Index: [],
  'read_segments':        offset  length  start_time  end_time
  0           0       1    0.001053  0.001068
  1           0       1    0.001072  0.001074
  2           0       1    0.001075  0.001076
  3           0       1    0.001076  0.001077
  4           0       1    0.001078  0.001079
  ...       ...     ...         ...       ...
  65524       0       1    0.064480  0.064480
  65525       0       1    0.064481  0.064481
  65526       0       1    0.064481  0.064482
  65527       0       1    0.064482  0.064483
  65528       0       1    0.064483  0.064484
  
  [65529 rows x 4 columns]}]